# Collecting description of offers

In [74]:
import pandas as pd
import requests
import numpy as np

In [75]:
file_name="CB_landplots_sale_2023-01-03.gzip"

In [76]:
df=pd.read_parquet(file_name, columns=["hash_id"])
df.head()

,hash_id
0,4122211404
1,1029104716
2,1148139084
3,779543628
4,259449932


In [77]:
desc_indices=df.hash_id
desc_indices

0       4122211404
1       1029104716
2       1148139084
3        779543628
4        259449932
           ...    
1402     331616348
1403    2859864156
1404    1967326044
1405    3826131804
1406    2561614684
Name: hash_id, Length: 1407, dtype: int64

In [78]:
desc_indices.drop_duplicates()
desc_indices.shape

(1407,)

In [79]:
collector_description={}

for i in range(len(desc_indices)):
    base_url = 'https://www.sreality.cz/api/cs/v2/estates/{}'.format(desc_indices[i])

    r = requests.get(base_url)

    #print("starting sleep")
    #sleep(randint(0,2))

    if r.status_code==404:
        break
    
    elif r.status_code==200:
        r_dict=r.json()
        r_dict["hash_id"]=desc_indices[i]

        collector_description[i]=r_dict
        
        print(f"Description {i+1} out of {len(desc_indices)} was scraped. hash_id: {desc_indices[i]}")

Description 1 out of 1407 was scraped. hash_id: 4122211404
Description 2 out of 1407 was scraped. hash_id: 1029104716
Description 3 out of 1407 was scraped. hash_id: 1148139084
Description 4 out of 1407 was scraped. hash_id: 779543628
Description 5 out of 1407 was scraped. hash_id: 259449932
Description 6 out of 1407 was scraped. hash_id: 1383523404
Description 7 out of 1407 was scraped. hash_id: 544662604
Description 8 out of 1407 was scraped. hash_id: 3787045212
Description 9 out of 1407 was scraped. hash_id: 1606944332
Description 10 out of 1407 was scraped. hash_id: 1390713180
Description 11 out of 1407 was scraped. hash_id: 2437322316
Description 12 out of 1407 was scraped. hash_id: 3477677644
Description 13 out of 1407 was scraped. hash_id: 2590434380
Description 14 out of 1407 was scraped. hash_id: 2976310348
Description 15 out of 1407 was scraped. hash_id: 194503756
Description 16 out of 1407 was scraped. hash_id: 991086940
Description 17 out of 1407 was scraped. hash_id: 49731

In [105]:
items_dict={"Zlevněno":"discounted", 
"Původní cena":"price_original", 
"ID zakázky":"id_order", 
"Aktualizace":"date_update", 
"Stavba":"building_type", 
"Stav objektu":"building_condition", 
"Umístění objektu":"building_location", 
"Užitná plocha":"area_net", 
"Plocha podlahová":"area_floor", 
"Celková cena":"price_total", 
"Poznámka k ceně":"price_note", 
"Doprava":"transport", 
"Plocha zastavěná":"area_build_up", 
"Datum nastěhování":"date_moving_in", 
"Rok kolaudace":"date_building_approval", 
"Vybavení":"equipped", 
"Elektřina":"electricity", 
"Komunikace":"traffic_communication", 
"Bezbariérový":"barrier_free", 
"ID":"id", 
#"Cena":"price", 
"Energetická náročnost budovy":"energy_efficient_rating", 
"Výtah":"lift", 
"Typ domu":"building_floor_type", 
"Podlaží":"building_floor", 
"Voda":"water", 
"Plyn":"gas", 
"Odpad":"waste", 
"Topení":"heating", 
"Telekomunikace":"telecommunication", 
"Rok rekonstrukce":"date_year_reconstruction", 
"Průkaz energetické náročnosti budovy":"energy_efficient_rating_card", 
"Stav":"availability", 
"Datum zahájení prodeje":"date_offer_start", 
"Ukazatel energetické náročnosti budovy":"energy_efficient_rating_index", 
"Datum prohlídky":"date_tour", 
"Datum prohlídky do":"date_tour_to", 
"Plocha zahrady":"area_garden",
"Výška stropu":"ceiling_height",
"Parkování":"parking",
"Cena za m²":"price_sqm",
"Počet kanceláří":"office_count",
"Garáž":"garage",
"Počet míst":"place_count",
"Bazén":"basin",
"Plocha pozemku":"area_landplot",
"Velikost podílu":"share_size",
"Vyvolávací cena":"auction_initial_price",
"Znalecký posudek":"expert_opinion_official",
"Minimální příhoz":"auction_min_bid", 
"Aukční jistina":"auction_principal",
"Druh dražby":"auction_type",
"Místo konání dražby":"auction_place",
"Datum konání dražby":"auction_date",
"Dražební vyhláška":"auction_decree",
"Posudek znalce":"expert_opinion"}

In [106]:
description_individual = {}
r_dict_names_all=pd.Series(dtype="object")
r_dict_types_all=pd.DataFrame(columns=["name", "type"])

for page, r_dict in collector_description.items():
        
        info_relevant = pd.Series(dtype="object")
        info_relevant["description"]=r_dict["text"]["value"]

        r_dict_values=pd.DataFrame(r_dict["items"], columns =['type', 'name', 'value'])

        r_dict_names=r_dict_values["name"]
        r_dict_names_all=r_dict_names_all.append(r_dict_names[~r_dict_names.isin(r_dict_names_all)])

        r_dict_types=r_dict_values[["type", "name"]]
        r_dict_types_all=pd.concat([r_dict_types_all, r_dict_types.loc[~r_dict_types["name"].isin(r_dict_types_all["name"]),:]])

        for name_raw in items_dict.keys():

                name_clean=items_dict[name_raw]

                if name_raw not in r_dict_names.values:
                        info_relevant[name_clean]=np.nan
                elif r_dict_values[r_dict_names==name_raw]["type"].all()=="set":
                        index_nr=int(r_dict_values.index[r_dict_values['name'] == name_raw].tolist()[0])
                        info_relevant[name_clean]=r_dict_values["value"][r_dict_values["name"]==name_raw][index_nr][0]["value"].values[0]
                        del index_nr
                else:
                        info_relevant[name_clean]=r_dict_values["value"][r_dict_values["name"]==name_raw].values[0]

        #if sum(r_dict_values['name'] == "Topení")>0:
        #        index_nr=int(r_dict_values.index[r_dict_values['name'] == "Topení"].tolist()[0])
        #        r_dict_values["value"][r_dict_values["name"]=="Topení"][index_nr][0]["value"]
        #        del index_nr

        description_individual[r_dict["hash_id"]] = info_relevant

/var/folders/s_/3ph0_hm1747ch5x8tpsf9d3r0000gn/T/ipykernel_69961/2050910403.py:13: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  r_dict_names_all=r_dict_names_all.append(r_dict_names[~r_dict_names.isin(r_dict_names_all)])
/var/folders/s_/3ph0_hm1747ch5x8tpsf9d3r0000gn/T/ipykernel_69961/2050910403.py:13: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  r_dict_names_all=r_dict_names_all.append(r_dict_names[~r_dict_names.isin(r_dict_names_all)])
/var/folders/s_/3ph0_hm1747ch5x8tpsf9d3r0000gn/T/ipykernel_69961/2050910403.py:13: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  r_dict_names_all=r_dict_names_all.append(r_dict_names[~r_dict_names.isin(r_dict_names_all)])
/var/folders/s_/3ph0_hm1747ch5x8tpsf9d3r0000gn/T/ipykernel_69961

In [107]:
r_dict_types_all

,name,type
0,Celková cena,price_czk
1,Cena za m²,price
2,ID zakázky,string
3,Aktualizace,edited
4,Umístění objektu,string
5,Plocha pozemku,area
2,Poznámka k ceně,string
6,Datum zahájení prodeje,date
7,Voda,set
8,Odpad,set


In [108]:
#r_dict_types_all
#boolean_formats=["boolean"]
#float_formats=["integer", "price_czk", "price_czk_old", "area", ""]

In [109]:
print("Add these values to your dictionary:")
print(r_dict_names_all[~r_dict_names_all.isin(items_dict.keys())])

Add these values to your dictionary:
0    Cena
dtype: object


In [110]:
df_desc = pd.concat(description_individual).unstack()
df_desc.head()

,description,discounted,price_original,id_order,date_update,building_type,building_condition,building_location,area_net,area_floor,...,share_size,auction_initial_price,expert_opinion_official,auction_min_bid,auction_principal,auction_type,auction_place,auction_date,auction_decree,expert_opinion
4122211404,Nabízíme Vám ve výhradním zastoupení ke koupi ...,NaN,NaN,2022-10-07-3,01.01.2023,NaN,NaN,Klidná část obce,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1029104716,Dovoluji si Vám exkluzivně nabídnout koupi sta...,NaN,NaN,XVA-IH-Pa5_46_7_3559,Dnes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1148139084,Hledáte vhodnou investiční příležitost jak zho...,NaN,NaN,SIM0317-0001,Dnes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
779543628,Nabízíme Vám ke koupi pozemek v katastrálním ú...,NaN,NaN,NaN,Dnes,NaN,NaN,NaN,NaN,NaN,...,1/4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259449932,Nabízíme Vám ke koupi pozemek v katastrálním ú...,NaN,NaN,NaN,Dnes,NaN,NaN,NaN,NaN,NaN,...,1/4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
df_desc["equipped"]=df_desc["equipped"].map({True: "ano", False: "ne", "Částečně":"částečně"}) 

### Saving data

In [112]:
df_desc.reset_index(inplace=True)
df_desc = df_desc.rename(columns = {'index':'hash_id'})
df_desc.to_parquet((file_name[:-5]+"_desc.gzip")) 

In [113]:
# end of final code

# New version of scraping - proxies and multiple requests at the same time

In [114]:
from fp.fp import FreeProxy
import gevent
from gevent import monkey, pool
#https://stackoverflow.com/questions/15322701/gevent-pool-with-nested-web-requests
#https://pypi.org/project/free-proxy/

In [115]:
proxy = FreeProxy(country_id=['DE', "CZ", "BR"]).get()

In [116]:
urls=np.add("https://www.sreality.cz/api/cs/v2/estates/", desc_indices)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U42'), dtype('int64')) -> None

In [ ]:
monkey.patch_all()

jobs = []
links = []
p = pool.Pool(10)

add

urls = [
    'http://www.google.com', 
    # ... another 100 urls
]
    
def get_jsons(url):
    r = requests.get(url)
    if r.status_code == 200:
        r_dict = r.json()
        r_dict["hash_id"]=desc_indices[i]

        collector_description[i]=r_dict
        
for url in urls:
    jobs.append(p.spawn(get_jsons, url))
gevent.joinall(jobs)